In [8]:
# Place the center of the map in an interesting area
center = (0.2923, 115.9698)

# Initialize a workflow map
m = wf.map.map
m.center = (38.7751, -121.7282) #
m.zoom = 12
# m.layout.width = '1200px'
# m.layout.height = '900'

dc = DrawControl(circlemarker={}, rectangle={}, polyline={})
wf.map.add_control(dc)

# Interactively compute deforested acreage 
pixel_sum = FullArray(wf.map, variable=dilated,
                     draw_control=dc)

output = ipywidgets.Output()
@output.capture()
def print_acreage(*args, **kwargs):
    # Estimate the surface area of one pixel in hectares
    pix_size = (156543.00/2**(max(m.zoom, 0)))**2 / 10000
    # Get the array equivalent to the polygon drawn by the user
    d = pixel_sum.data            
    # Compute deforested and total acreage
    deforested = round((d==1).sum() * pix_size, 2)
    total = round(((d==1) | (d==0)).sum() * pix_size, 2)
    print("Deforested area within AOI : \n\n{} ha (of {} ha in total)".format(deforested, total))

# Perform the acreage computation
dc.on_draw(print_acreage)

# Print acreage in a clear button on the map in the top right corner
output_clear_button = ipywidgets.Button(
    description="Clear Output Window", layout=ipywidgets.Layout(width="auto"))
output_clear_button.on_click(lambda b: output.clear_output())
output_vbox = ipywidgets.VBox([output, output_clear_button])
m.add_control(ipyleaflet.WidgetControl(widget=output_vbox, position='bottomleft'))

ipywidgets.HBox([wf.map])

NameError: name 'dilated' is not defined

In [7]:
from ipyleaflet import DrawControl
from descarteslabs import workflows as wf
import descarteslabs as dl


class FullArray(object):
    def __init__(self, map_widget, variable=None, draw_control=None):
        self.m = map_widget
        if draw_control is None:
            self.draw_control = DrawControl()
            self.draw_control.circle = {}
            self.draw_control.polygon = {}
            self.draw_control.circlemarker = {}
            self.draw_control.polyline = {}
            self.draw_control.rectangle = {
                "shapeOptions": {
                    "fillColor": "#000000",
                    "color": "#fca45d",
                    "fillOpacity": 0.0,
                }
            }
            self.m.add_control(self.draw_control)
        else:
            self.draw_control = draw_control
        self.variable = variable
        self.draw_control.on_draw(self.calculate)

    def calculate(self, *args, **kwargs):
        last_draw = self.draw_control.last_draw

        if last_draw["geometry"]["type"] == "Point":
            last_draw_context = wf.GeoContext.from_dltile_key(
                dl.raster.dltile_from_latlon(
                    self.draw_control.last_draw["geometry"]["coordinates"][1],
                    self.draw_control.last_draw["geometry"]["coordinates"][0],
                    156543.00 / 2 ** (max(self.m.zoom, 0)),
                    2,
                    0,
                ).properties.key
            )

        elif last_draw["geometry"]["type"] == "Polygon":
            last_draw_context = wf.GeoContext(
                geometry=last_draw["geometry"],
                resolution=156543.00 / 2 ** (max(self.m.zoom, 0)),
                crs="EPSG:3857",
                bounds_crs="EPSG:4326",
            )

        data = self.variable.compute(last_draw_context)
        self._data = data.ndarray

    _data = None

    @property
    def data(self):
        if self._data is not None:
            return self._data
        else:
            raise RuntimeError(
                "To display the deforested acreage please draw a polygon first"
            )

In [4]:
from descarteslabs import workflows as wf
import ipyleaflet
import ipywidgets
import matplotlib.pyplot as plt
import descarteslabs as dl
import descarteslabs.workflows as wf
from shapely import geometry as sgeom
import logging
import will_utils as utils
from ipyleaflet import DrawControl
logging.basicConfig(level=logging.INFO)